In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
import requests
import zipfile
import io
import matplotlib.pyplot as plt

data_url = 'https://github.com/Griselia23/wastemanagementapk/raw/main/ML/dataset/trash.zip'

# Download dan extract data
response = requests.get(data_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall('trash_dataset')

data_dir = 'trash_dataset/dataset-resized'

In [2]:
#dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(512, 384),
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(512, 384),
    batch_size=32
)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [3]:
# proses make cnn
image_input = Input(shape=(512, 384, 3))
conv1 = Conv2D(16, (3, 3), activation='relu')(image_input)
pool1 = MaxPooling2D(2, 2)(conv1)
conv2 = Conv2D(32, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(2, 2)(conv2)
conv3 = Conv2D(64, (3, 3), activation='relu')(pool2)
pool3 = MaxPooling2D(2, 2)(conv3)
flatten = Flatten()(pool3)
image_output = Dense(256, activation='relu')(flatten)


In [4]:
# modelnya
model = Model(inputs=image_input, outputs=image_output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
# nyoba latih model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 256)

In [ ]:
# liat akurasi model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
//save model with .tflife